In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("vgsales.csv")

print("=" * 60)
print("INITIAL DATA OVERVIEW")
print("=" * 60)
print(f"Shape: {df.shape[0]} rows × {df.shape[1]} columns\n")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())
print("\nBasic statistics:")
print(df.describe())
print("\nMissing values:")
print(df.isnull().sum())

INITIAL DATA OVERVIEW
Shape: 16598 rows × 11 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB
None

First 5 rows:
   Rank                      Name Platform    Year         Genre Publisher  \
0     1                Wii Sports      Wii  2006.0        Sports  Nintendo   
1     2         Super Mario Bros.      N

In [3]:
print("\n" + "=" * 60)
print("HANDLING MISSING VALUES")
print("=" * 60)

print(f"\nBefore: {len(df)} rows")
print(f"Missing Year: {df['Year'].isnull().sum()}")
print(f"Missing Publisher: {df['Publisher'].isnull().sum()}")

df = df.dropna(subset=["Year"])
print(f"After dropping missing Year: {len(df)} rows")


HANDLING MISSING VALUES

Before: 16598 rows
Missing Year: 271
Missing Publisher: 58
After dropping missing Year: 16327 rows


In [5]:
remaining_null_pub = df["Publisher"].isnull().sum()
print(f"Remaining missing Publisher: {remaining_null_pub}")

df["Publisher"] = df["Publisher"].fillna("Unknown")

print(f"After filling Publisher: {df['Publisher'].isnull().sum()} missing")

Remaining missing Publisher: 36
After filling Publisher: 0 missing


In [9]:
print("\n" + "=" * 60)
print("HANDLING DUPLICATES")
print("=" * 60)

print(f"Exact duplicates: {df.duplicated().sum()}")
print(f"Same Name+Platform: {df.duplicated(subset=['Name', 'Platform']).sum()}")
dupes = df[df.duplicated(subset=["Name", "Platform"], keep=False)].sort_values(
    ["Name", "Platform"]
)
print("\nDuplicate entries:")
print(dupes[["Rank", "Name", "Platform", "Year", "Publisher", "Global_Sales"]])


HANDLING DUPLICATES
Exact duplicates: 0
Same Name+Platform: 3

Duplicate entries:
        Rank                         Name Platform    Year        Publisher  \
603      604                Madden NFL 13      PS3  2012.0  Electronic Arts   
16127  16130                Madden NFL 13      PS3  2012.0  Electronic Arts   
5900    5902  Need for Speed: Most Wanted       PC  2005.0  Electronic Arts   
11676  11678  Need for Speed: Most Wanted       PC  2012.0  Electronic Arts   
1173    1175  Need for Speed: Most Wanted     X360  2012.0  Electronic Arts   
1530    1532  Need for Speed: Most Wanted     X360  2005.0  Electronic Arts   

       Global_Sales  
603            2.56  
16127          0.01  
5900           0.30  
11676          0.08  
1173           1.58  
1530           1.29  


In [11]:
before_count = len(df)
df = df.sort_values("Global_Sales", ascending=False)
df = df.drop_duplicates(subset=["Name", "Platform", "Year"], keep="first")
df = df.sort_values("Rank").reset_index(drop=True)
after_count = len(df)

print(f"\nRemoved {before_count - after_count} true duplicates (same name+platform+year)")
print(f"Rows remaining: {after_count}")



Removed 1 true duplicates (same name+platform+year)
Rows remaining: 16326


In [13]:
print("\n" + "=" * 60)
print("DATA TYPE CORRECTIONS")
print("=" * 60)

# Year is float64 due to NaN — convert to int now that NaNs are gone
df["Year"] = df["Year"].astype(int)

print(f"Year dtype: {df['Year'].dtype} (was float64)")
print(f"Year range: {df['Year'].min()} – {df['Year'].max()}")


DATA TYPE CORRECTIONS
Year dtype: int64 (was float64)
Year range: 1980 – 2020


In [17]:
df["Decade"] = (df["Year"] // 10 * 10).astype(str) + "s"
print(f"\nDecade distribution:\n{df['Decade'].value_counts().sort_index()}")



Decade distribution:
Decade
1980s     205
1990s    1769
2000s    9208
2010s    5143
2020s       1
Name: count, dtype: int64


In [19]:
df["NA_Share"] = (df["NA_Sales"] / df["Global_Sales"] * 100).round(2)
df["EU_Share"] = (df["EU_Sales"] / df["Global_Sales"] * 100).round(2)
df["JP_Share"] = (df["JP_Sales"] / df["Global_Sales"] * 100).round(2)
df["Other_Share"] = (df["Other_Sales"] / df["Global_Sales"] * 100).round(2)

print("\nRegional sales share (mean %):")
print(f"  NA:    {df['NA_Share'].mean():.1f}%")
print(f"  EU:    {df['EU_Share'].mean():.1f}%")
print(f"  JP:    {df['JP_Share'].mean():.1f}%")
print(f"  Other: {df['Other_Share'].mean():.1f}%")


Regional sales share (mean %):
  NA:    45.4%
  EU:    22.9%
  JP:    24.4%
  Other: 6.5%


In [21]:
hit_threshold = df["Global_Sales"].quantile(0.75)
df["Is_Hit"] = (df["Global_Sales"] >= hit_threshold).astype(int)

print(f"\nHit threshold (75th percentile): {hit_threshold}M")
print(f"Hit games: {df['Is_Hit'].sum()} ({df['Is_Hit'].mean()*100:.1f}%)")
print(f"Non-hit games: {(df['Is_Hit'] == 0).sum()} ({(1 - df['Is_Hit'].mean())*100:.1f}%)")
# a game is mark as "hit" if its sale is in the top 25%


Hit threshold (75th percentile): 0.48M
Hit games: 4092 (25.1%)
Non-hit games: 12234 (74.9%)


In [23]:
publisher_sales = df.groupby("Publisher")["Global_Sales"].sum()

def classify_publisher(publisher):
    total = publisher_sales.get(publisher, 0)
    if total >= 150:
        return "AAA"
    elif total >= 10:
        return "Mid"
    else:
        return "Indie"

df["Publisher_Tier"] = df["Publisher"].apply(classify_publisher)

print(f"\nPublisher tier distribution:")
print(df["Publisher_Tier"].value_counts())
print(f"\nMean global sales by tier:")
print(df.groupby("Publisher_Tier")["Global_Sales"].mean().round(3))


Publisher tier distribution:
Publisher_Tier
AAA      8672
Mid      4119
Indie    3535
Name: count, dtype: int64

Mean global sales by tier:
Publisher_Tier
AAA      0.769
Indie    0.140
Mid      0.403
Name: Global_Sales, dtype: float64


In [25]:
generation_map = {
    # Generation 2-3 (Pre-NES era)
    "2600": "Gen 2-3",
    # Generation 3 (8-bit)
    "NES": "Gen 3", "SG": "Gen 3",
    # Generation 4 (16-bit)
    "SNES": "Gen 4", "GEN": "Gen 4", "TG16": "Gen 4",
    # Generation 5 (32/64-bit)
    "PS": "Gen 5", "N64": "Gen 5", "SAT": "Gen 5",
    # Generation 6
    "PS2": "Gen 6", "XB": "Gen 6", "GC": "Gen 6", "DC": "Gen 6",
    # Generation 7
    "PS3": "Gen 7", "X360": "Gen 7", "Wii": "Gen 7",
    # Generation 8
    "PS4": "Gen 8", "XOne": "Gen 8", "WiiU": "Gen 8",
    # Handhelds
    "GB": "Handheld", "GBA": "Handheld", "DS": "Handheld",
    "3DS": "Handheld", "PSP": "Handheld", "PSV": "Handheld", "GG": "Handheld",
    # PC & Other
    "PC": "PC",
    "NG": "Other", "WS": "Other", "3DO": "Other", "PCFX": "Other",
}
df["Platform_Gen"] = df["Platform"].map(generation_map).fillna("Other")

print(f"\nPlatform generation distribution:")
print(df["Platform_Gen"].value_counts())


Platform generation distribution:
Platform_Gen
Handheld    5151
Gen 7       3828
Gen 6       3524
Gen 5       1678
PC           943
Gen 8        692
Gen 4        268
Gen 2-3      116
Gen 3         98
Other         28
Name: count, dtype: int64


In [27]:
print("\n" + "=" * 60)
print("FINAL DATASET SUMMARY")
print("=" * 60)

print(f"\nShape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\nColumns: {list(df.columns)}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nData types:\n{df.dtypes}")

assert df.isnull().sum().sum() == 0, "ERROR: Still have missing values!"
assert df["Year"].dtype == int, "ERROR: Year is not int!"
# Note: Regional shares may not sum to exactly 100% due to rounding in the
# source data, especially for low-sales titles. This is expected and acceptable.
share_sums = df["NA_Share"] + df["EU_Share"] + df["JP_Share"] + df["Other_Share"]
print(f"   Regional share sums: mean={share_sums.mean():.1f}%, median={share_sums.median():.1f}%")
print(f"   (Minor deviations from 100% are due to rounding in source data)")

print("\n✅ All validation checks passed!")


FINAL DATASET SUMMARY

Shape: 16326 rows × 19 columns

Columns: ['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Decade', 'NA_Share', 'EU_Share', 'JP_Share', 'Other_Share', 'Is_Hit', 'Publisher_Tier', 'Platform_Gen']

Missing values:
Rank              0
Name              0
Platform          0
Year              0
Genre             0
Publisher         0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
Decade            0
NA_Share          0
EU_Share          0
JP_Share          0
Other_Share       0
Is_Hit            0
Publisher_Tier    0
Platform_Gen      0
dtype: int64

Data types:
Rank                int64
Name               object
Platform           object
Year                int64
Genre              object
Publisher          object
NA_Sales          float64
EU_Sales          float64
JP_Sales          float64
Other_Sales       float64
Global_Sales      flo

In [29]:
output_path = "vgsales_cleaned.csv"
df.to_csv(output_path, index=False)
print(f"\n📁 Cleaned dataset saved to: {output_path}")
print(f"   Original: 16,598 rows × 11 columns")
print(f"   Cleaned:  {df.shape[0]} rows × {df.shape[1]} columns")
print(f"   New features: Decade, NA/EU/JP/Other_Share, Is_Hit, Publisher_Tier, Platform_Gen")


📁 Cleaned dataset saved to: vgsales_cleaned.csv
   Original: 16,598 rows × 11 columns
   Cleaned:  16326 rows × 19 columns
   New features: Decade, NA/EU/JP/Other_Share, Is_Hit, Publisher_Tier, Platform_Gen
